In [1]:
"""
Created on Fri Jan 12 18:33:56 2018
@author: ruben
"""
import io
import time
import pickle
import gzip
from opencc import OpenCC
ocs2t = OpenCC('s2t')
oct2s = OpenCC('t2s')

In [31]:
# Read English dobj-collocation pairs
start_time = time.time()

folder = '/mnt/c/NLP/collo/'

depen_reln = ('dobj', 'amod', 'nn')[0]  # select an appropriate dependency relationship

pickle_in = gzip.open( folder + depen_reln + ".jj.zhs.gz", "rb")
JJ_ZH = pickle.load(pickle_in)
pickle_in.close()

pickle_in = gzip.open( folder + depen_reln + ".nn.zhs.gz", "rb")
NN_ZH = pickle.load(pickle_in)
pickle_in.close()

pickle_in = gzip.open( folder + depen_reln + ".jj.en.gz", "rb")
JJ_EN = pickle.load(pickle_in)
pickle_in.close()

pickle_in = gzip.open( folder + depen_reln + ".nn.en.gz", "rb")
NN_EN = pickle.load(pickle_in)
pickle_in.close()

elapsed_time = round(time.time() - start_time, 2)
print("JJ_ZH, NN_ZH, JJ_EN and NN_EN took", elapsed_time, " seconds to build")


JJ_ZH, NN_ZH, JJ_EN and NN_EN took 6.05  seconds to build


In [3]:
depen_reln

'dobj'

In [4]:
len(JJ_EN), len(NN_EN), len(JJ_ZH), len(NN_ZH)

(25152, 6032, 64167, 25975)

In [5]:
words = '''
凝聚力 壓制 主權 必然 肯定 必須 發揮 
'''
w = words.strip().split()[-1]
#' | '.join(zh2en(words[-1], 35))
w = oct2s.convert(w)
for x in JJ_ZH[oct2s.convert(w)]: print(ocs2t.convert(x), '| ', end='')

保證 | 促進 | 影響 | 認識 | 聯 | 限制 | 失去 | 改進 | 充實 | 作 | 協助 | 傳播 | 有助於 | 危及 | 妨礙 | 支持 | 確保 | 破壞 | 重視 | 包括 | 加強 | 需要 | 有所 | 加以 | 服務 | 阻礙 | 得到 | 

In [6]:
words = '''
hole sovereignty  
'''
w = words.strip().split()[-1]#for x in NN_EN['supervise']: print x,
for x in JJ_EN[w]: print(x, '| ', end='')
#print len(NN_EN)

offend | reclaim | achieve | declare | prevent | void | concern | preserve | relegate | wish | compromise | put | change | safeguard | keep | emphasize | possess | contest | regard | maintain | reject | solve | secure | have | limit | grant | remove | proclaim | express | endanger | explain | share | recover | reiterate | impair | advocate | ensure | consolidate | gain | underpin | protect | include | question | extend | confirm | acknowledge | relinquish | defend | curtail | recognize | accept | reaffirm | balance | assault | article | alienate | hold | reinforce | enjoy | infringe | surrender | attain | usurp | reconfirm | forgo | know | suspend | exert | say | conceptualize | stress | set | waive | honour | erode | enshrine | diminish | lose | implement | ignore | adjudicate | assert | challenge | affect | subvert | provide | want | cite | assume | impose | breach | prejudice | negotiate | nullify | promote | threaten | reassume | undermine | bear | restore | involve | expand | dero

In [7]:
from multivec import BilingualModel, MonolingualModel

In [8]:
start_time = time.time()
from multivec import MonolingualModel, BilingualModel
MODEL = ['lemma', 'os', 'un16m'][0]
mode = None; model_en = None; model_zh = None
if MODEL == 'lemma':
    model = BilingualModel(b'/mnt/c/NLP/collo/un4m.lemma.bin')
    model_en = model.src_model
    model_zh = model.trg_model
elif MODEL == 'os':
    model = BilingualModel('oscleaned.en-zh.bin')
elif MODEL == 'un16m':
    model = BilingualModel('un16m.bin')
    
elapsed_time = round(time.time() - start_time, 2)
print("Multivec bilingual and monolingal models took ", elapsed_time, " seconds to load")


Multivec bilingual and monolingal models took  67.15  seconds to load


In [9]:
model.trg_closest?

Docstring: <no docstring>
Type:      builtin_function_or_method


In [10]:
def getCollocates(v, n, k): # 英翻中

    v = v.encode(); n = n.encode()
    base_trg = [w.decode('utf8') for (w,s) in model.trg_closest(n, 8)]
    k = 0
    candidates = set()
    for b in base_trg:
        b = oct2s.convert(b)
        if b in JJ_ZH:
            colloSize = len(JJ_ZH[b])
            print(ocs2t.convert(b), ' - no. of collocations:', colloSize)
            if colloSize > 2000:
                colloSize = 2000
            for col in list(JJ_ZH[b])[0:colloSize]:
                col = ocs2t.convert(col)
                col = col.encode("utf8")
                #print "| col", col,
                try:
                    sim = model.similarity(v, col)
                    if sim >= 0.40:
                        k += 1
                        candidates.update([col.decode('utf8')])
                except:
                    #print "* v0", "not found"
                    continue
    return (base_trg, candidates, k)


In [11]:
start_time = time.time()
v = 'negotiate'; n = 'freedom'; 
v = 'threaten'; n = 'rule'
v = 'contemplate'; n = 'presence'
v = 'spearhead'; n = 'movement'
v = 'secure'; n = 'majority'
#v = 'compromise'; n = 'position'
#v = 'convince'; n = 'veracity'
#v = 'assume'; n = 'responsibility'
#v = 'shirk'; n = 'responsibility'
#v = 'fair'; n = 'game'
v = ''; n = ''
v = 'combat'; n = 'crime'
v = 'oppress'; n = 'freedom'
v = 'cause'; n = 'trouble'
v = 'deliver'; n = 'ruling'
v = 'appoint'; n = 'official'
k = -1
(base_trg, candidates, k) = getCollocates(v, n, k)
print("k =", k)
elapsed_time = round(time.time() - start_time, 2)
print("\n* Collocation candidates look-up:", elapsed_time, "sec")
print("* Bases:")
print(', '.join([ocs2t.convert(b) for b in base_trg]))
print("* Collocates:")
print(', '.join([ocs2t.convert(b) for b in candidates]))

官員  - no. of collocations: 602
官員們  - no. of collocations: 13
公務  - no. of collocations: 45
高官  - no. of collocations: 5
官方  - no. of collocations: 88
要員  - no. of collocations: 2
接待費  - no. of collocations: 1
k = 8

* Collocation candidates look-up: 0.38 sec
* Bases:
官員, 官員們, 公務, 高官, 官方, 要員, 正式, 接待費
* Collocates:
聘任, 委派, 指定, 委任, 提名, 任命, 任, 指派


In [12]:
[w for w in candidates]

['聘任', '委派', '指定', '委任', '提名', '任命', '任', '指派']

In [13]:
for x in JJ_ZH['暴乱']: print(ocs2t.convert(x), '| ', end='')

驅散 | 利用 | 準備 | 禁止 | 震撼 | 受到 | 應付 | 直指 | 中的 | 提醒 | 返回 | 爆發 | 犯有 | 逃離 | 陷入 | 起因於 | 激起 | 製造 | 引起 | 制止 | 篡奪 | 發生 | 進行 | 反 | 察 | 有 | 發動 | 接受 | 殺害 | 對在 | 減少 | 參與 | 譴責 | 處理 | 鎮壓 | 稱爲 | 反擊 | 導致 | 懲罰 | 佔 | 出現 | 開展 | 武裝 | 歡迎 | 造成 | 表示 | 襲擊 | 參加 | 終止 | 成爲 | 煽動 | 發向 | 來自 | 

In [33]:
words = '''
壓倒性 打擊 過度 美中不足 證實 無數 很多 誤差 計算 熱衷 變化 自製 主流 持續 
改革 精確 整理 執著  丈量 推翻 遺憾 煎熬 流傳 於世 後世 模仿 壓制 因此 貢獻 
激勵 鼓舞 喜悅 顛沛  流離 開宗明義 發揮 規範 希望 給 圖像 規範 充滿 過程 推導 
不道德 改進 完整性 付之一炬 否認 叛逆 細膩 思考 敏銳 外快 主張 不滿 仇視 無奈 
拘束 委託 基準 共同體 例示 果然 所作所為 論述 作風 傑出 正直 珍貴 豎立 榜樣 
損耗  耗損 強勁 消耗 模糊 推動力 否定 排斥 招架 
'''
w = words.strip().split()[-1]
#' | '.join(zh2en(words[-1], 35))
w = oct2s.convert(w)
for (x, s) in model.src_closest(w.encode(), 30): print(x.decode(), '| ', end='')

dare | overwhelmed | speculate | canvass | fiercely | quietly | frenzy | impatience | able | fend | yesterday | unable | afraid | mad | collegially | relive | lavish | scalp | unsuccessfully | tired | disloyal | misfortune | unending | preoccupy | recount | Greed | alone | overwhelm | grip | inability | 

In [15]:
for x in JJ_ZH[w]: print(ocs2t.convert(x), '| ', end='')

動盪 | 認識 | 予以 | 受到 | 搞 | 面對 | 限制 | 戰勝 | 鼓勵 | 發生 | 進行 | 反 | 加劇 | 有 | 減少 | 無 | 處理 | 消除 | 確保 | 防止 | 存在 | 帶來 | 包括 | 缺乏 | 導致 | 遭受 | 害怕 | 有所 | 結束 | 反對 | 免於 | 訂立 | 免遭 | 舉辦 | 出現 | 受 | 加以 | 作爲 | 產生 | 造成 | 減輕 | 遭 | 表現 | 沒有 | 遭到 | 注意 | 克服 | 發表 | 相對於 | 製造 | 

In [16]:
words = '''
cease entail contain enthusiastic contribution prevailing prevalent subversive 
revolutionary radical phenomenon famous known continuously attraction measure 
regulation stipulate suffering agonizing desire invent conjecture legacy suppress 
deny obtain poverty confirm model offer theoretical derive burn retract conformance 
protection improved trolley questionable action move advocate style policy 
ego  referee judge  fallacy  theorize condition forget fact truth topic theme 
offender offense abomination struggle upright banish promote infinite blame explain 
discuss divorced revolutionary phenomena manifestation propose drift contrition 
attrition friction depletion blurred obsession adhere turmoil style 
'''
w = words.strip().split()[-1]
#' | '.join(zh2en(words[-1], 35))
for (x, s) in model_en.closest(w.encode(), 30): print(x.decode(), '| ', end='')

script | lifestyle | mentality | mindedness | mode | rote | tone | metaphor | mindset | game | subtle | monolithic | mixture | paradigm | collegial | confrontational | format | saxon | slogan | hybrid | dance | modern | joke | behaviour | geometric | hobby | classical | spoken | unconventional | materialistic | 

In [17]:
words = '''
manipulate fluctuate motivation contagious trick priming mentality mindset 
experimenter attractive stressful prefer physical measuring  inspiring rapport 
desire legacy relevance relevant official outlaw illegitimate illegal retract 
trolley cablecar desperate trolley careen action theorize theoretical 
'''
w = words.strip().split()[-1]
#' | '.join(zh2en(words[-1], 35))
for (x, s) in model.trg_closest(w.encode(), 30): print(ocs2t.convert(x.decode()), '| ', end='')

理論 | 理論性 | 純理論 | 原理 | 實踐 | 概念上 | 哲學 | 實用 | 理論化 | 概念 | 學說 | 內涵 | 推理 | 抽象 | 方法論 | 學術性 | 法律學 | 概念性 | 數學 | 法理 | 法學 | 實際 | 純粹 | 學派 | 社會學 | 理論課 | 法律性 | 描述性 | 方法學 | 經驗性 | 

In [18]:
def getCollocates2(v, n, k, simThreshold=0.35): # 中翻英
    '''
    v, n: Chinese, zh-cn, UTF-8
    '''
    
    vs = oct2s.convert(v).encode() # convert to zh-tw, and then to bytecode before passing to library
    #print("vs=", vs)
    ns = oct2s.convert(n).encode()
    #print("ns=", ns)
    base_src = [w.decode('utf8') for (w,s) in model.src_closest(ns, 10)]
    k = 0
    candidates = set()
    for b in base_src:
        if b in JJ_EN:
            colloSize = len(JJ_EN[b])
            print(b, '- no. of collocations:', colloSize)
            #colloSize = min(colloSize, 3000)
            for col in list(JJ_EN[b])[0:colloSize]:
                #print "| col", col,
                try:
                    sim = model.similarity(col.encode(), vs)
                    if sim >= simThreshold:
                        k += 1
                        candidates.update([col])
                except:
                    #print "* v0", "not found"
                    continue
    return (base_src, candidates, k)

In [41]:
start_time = time.time()


#v = '幫助'; n = '成長';
#v = '侵犯'; n = '權利';
v = '減低'; n = '稅收';
v = '過度'; n = '打擊';
v = '流血'; n = '衝突'
v = '肩負'; n = '責任'
v = '大幅度'; n = '增加'
v = '侵犯'; n = '權益';
v = '侵犯'; n = '隱私';
v = '前進'; n = '戰場';
v = '健全'; n = '基礎';
v = '遵守'; n = '程序';
v = '維護'; n = '利益';
v = '承擔'; n = '責任';
v = '損害'; n = '名譽';
#v = '通過'; n = '憲法';
v = '減輕'; n = '負擔'; 
v = '執行'; n = '法律';
v = '實現'; n = '夢想';
v = '實現'; n = '願望';
v = '交換'; n = '角色';
v = '防止'; n = '騷亂';
v = '保障'; n = '人權';
v = '通過'; n = '憲法';
v = '通過'; n = '條例';
v = '制訂'; n = '規範';
v = '超越'; n = '目標';
#v = '達成'; n = '目標';
#v = '制訂'; n = '法律';
v = '持續'; n = '成長';  # sustain
v = ''; n = ''
v = '引起'; n = '恐慌'
v = '補償'; n = '遺憾'
v = '鼓起'; n = '勇氣'
v = '改善'; n = '缺失'
v = '抒解'; n = '恐懼'
v = '保障'; n = '權益';
v = '打擊'; n = '犯罪'; 
v = '放下'; n = '身段'; 
v = '殘酷'; n = '現實'; 
v = '展現'; n = '風度'; 
v = '推卸'; n = '責任';

k = -1
(base_src, candidates, k) = getCollocates2(v, n, k, simThreshold=0.25)
base_src = [ele for ele in base_src]
print("k =", k)
elapsed_time = round(time.time() - start_time, 2)
print("\n* Collocation candidates look-up:", elapsed_time, "sec")
print("* Bases:     \n", ' ', end=''); print(" ", ', '.join(base_src))
print("* Collocates:\n", ' ', end=''); print(" ", ', '.join(list(candidates)))


responsibility - no. of collocations: 485
liability - no. of collocations: 180
abdication - no. of collocations: 5
onus - no. of collocations: 13
shoulder - no. of collocations: 11
duty - no. of collocations: 243
accountable - no. of collocations: 1
obligation - no. of collocations: 442
k = 131

* Collocation candidates look-up: 0.33 sec
* Bases:     
    responsibility, liability, abdication, onus, shirk, shoulder, duty, accountable, obligation, criminally
* Collocates:
    shirk, vest, circumvent, exonerate, elude, betray, dilute, disclaim, defy, supplant, discharge, overlook, transcend, relegate, disregard, obscure, devolve, assume, refuse, bestow, trigger, camouflage, arrogate, rest, relieve, abandon, enjoin, skirt, usurp, deflect, fail, excuse, fulfill, disown, ascribe, shun, flout, fulfil, cede, forget, preclude, entrust, overburden, override, extinguish, negate, do, subordinate, escape, sidestep, renounce, waive, repudiate, evade, reassert, abdicate, diminish, obey, impute, invo

In [29]:
' | '.join(candidates)

'lamentable | painful | tough | many | outrageous | genocidal | palestinian | cruel | eternal | deep | sad | lethal | cuban | tragic | naked | alarming | colonialist | perilous | monolithic | unjust | brute | emotional | reactionary | harsh | unfortunate | gruesome | terrible | distorted | severe | bleak | brutal | troubling | oppressive | grim | archaic | striking | cold | utopian | disturbing | widespread | atrocious | horrific | apocalyptic | perverse | most | profound | hidden | distressing | dramatic | serious | appalling | discouraging | shameful | shocking | combative | numerous | diseased | pervasive | reformist | totalitarian | terrifying | dark | sombre | dire | sordid | unacceptable | grave | unwelcome | contemporary | vivid | horrible | colonial | biased | prejudiced | conspicuous | excessive | devastating | dangerous | very | stark | illegal | sobering | past | bitter | prevalent | wrong | inhuman | barbaric | frustrating | racist | extremist | bad | eritrean'

In [39]:
w = 'credibility'
w = 'legitimacy'; w = 'cynic'
w = 'permeate'; w = 'tergiversate'; 
w = 'eternal'
w = 'shirk'
for (x,s) in model.trg_closest(w.encode('utf8'), 8): print(ocs2t.convert(x.decode('utf8')), '| ', end='')

推卸 | 盡到 | 責任 | 責無旁貸 | 義不容辭 | 迴避 | 有責任 | 責怪 | 

In [22]:
v

'打擊'

In [38]:
w = '壓倒性'; w = '打擊'; w = u'犯罪'; w = u'報到'; w = u'别有用心'; w = u'紛亂'; w = u'凌駕'
w = '穩健'; w = '款項'; w = u'封建'; w = u'基礎'; w = u'變遷'; w = u'動態'; w = u'鞭策'; w = u'督促'
w = '永恆'; w = '法則' #亙古不變
w = '穩健'; w = '維護'; w = u'檢舉'
w = oct2s.convert(w)
for (x, s) in model.src_closest(w.encode("utf8"), 15): print(x.decode('utf8'))

complaint
investigate
investigating
prosecute
investigation
inquiry
prosecution
violator
investigatory
alleged
informant
detective
incriminated
illtreatment
maltreatment


In [37]:
w = 'appropriate'
w = 'delicacy'
w = 'bureaucracy'
w = 'upheaval'
w = 'perturbation'
w = 'sophisticated'
w = 'cause'
w = 'vacate'
w = 'permeate'
for (x,s) in model.trg_closest(w.encode('utf8'), 15): print(ocs2t.convert(x.decode('utf8')))

貫穿
深植
方方面面
紮根於
無處不在
兩極分化
植根於
深入人心
滲透
遍及
根於
形形色色
深深
心中
分化
